In [1]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

os.environ['OPENAI_API_KEY']= os.getenv("OPENAI_API_KEY")

In [2]:

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("bestbook.pdf")
docs = loader.load()

In [3]:
#split the docs into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
textsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
document = textsplitter.split_documents(docs)

In [9]:
document[:500]

[Document(page_content='Praise\tfor\t\nBecoming\tSupernatural\n“From\ta\tgifted\tscientist\tand\tpassionate\tteacher,\tthis\tunique\tand\tpractical\tguide\nshows\tus—step\tby\tstep—how\tto\tmove\tbeyond\tthe\tlimits\tof\tthe\tknown\tand\tinto\tan\nextraordinary\tnew\tlife.”\n—\tTony\tRobbins\n,\t#1\t\nNew\tYork\tTimes\n\tbest-selling\tauthor\tof\t\nUnshakeable\n“I\thave\tlong\tbeen\ta\tfan\tof\tDr.\tJoe\tDispenza’s\twork.\tIn\n\tBecoming\tSupernatural\n,\nyou\twill\tlearn\texactly\thow\tyou\tcan\ttranscend\tthe\tlimitations\tof\tyour\tpast—\nincluding\thealth\tchallenges—and,\tquite\tliterally,\tcreate\ta\tnew\tbody,\ta\tnew\tmind,\nand\ta\tnew\tlife.\tThis\tinformation\tis\tthrilling,\tlife\tchanging,\tand\tincredibly\npractical.”\n—\tChristiane\tNorthrup,\tM.D.\n,\t\nNew\tYork\tTimes\n\tbest-selling\tauthor\tof\nGoddesses\tNever\tAge\n“Dr.\tJoe\tDispenza\tis\ta\tdoctor,\ta\tscientist,\tand\ta\tmodern-day\tmystic.\t.\t.\t.\tIn\ta\nstyle\tthat\tis\tsimple,\tstraightforward,\tand\teasy\

In [8]:
#convert word2vec using embedding and store as vecdb

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(document[:500],OpenAIEmbeddings())


In [10]:
#basic search to compare afterwards with retriver results
query="how to meditate"

result=db.similarity_search(query)
result[0].page_content


'Tuning\tIn\tto\tNew\tPotentials\tMeditation\nStart\tby\tresting\tyour\tattention\tin\tdifferent\tparts\tof\tyour\tbody\tas\twell\tas\tthe\nspace\taround\tthose\tparts\tof\tyour\tbody.\t(You\twill\tlearn\tmore\tabout\tdoing\tthis\tand\nwhy\tit’s\timportant\tin\tthe\tnext\tchapter,\tbut\tfor\tnow,\tit’s\tenough\tto\tknow\tthat\nfocusing\ton\tthe\tspace\taround\tyour\tbody\thelps\tchange\tyour\tbrain\twaves,\tmoving\nyou\tfrom\tan\tincoherent\tbeta\tbrainwave\tpattern\tto\ta\tcoherent\talpha\tand\ttheta\nbrain-wave\tpattern).\tBecome\taware\tof\tthe\tinfinite,\tvast\tspace\tway\tout\tbehind\nyour\teyes\tin\tthis\teternal\tblack\tspace,\tthe\tspace\tin\tthe\tcenter\tof\tyour\thead,\tthe\nspace\tbetween\tthe\tback\tof\tyour\tthroat\tand\tthe\tback\tof\tyour\thead,\tand\tthen\nbeyond\tyour\thead\tin\tspace.\tThen\tmove\tto\tbecoming\taware\tof\tthe\tspace\tin\tthe\ncenter\tof\tyour\tthroat,\tthe\tspace\tbeyond\tyour\tthroat\tand\taround\tyour\tneck,\tthe\nspace\tin\tthe\tcenter\tof\tyour\tc

In [12]:
#retriver to get relavant info from unstructred query from user

retriever =db.as_retriever()

In [15]:
#aking prompts that is empty of relavant and user query which will be filled later

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('''
answer the question only based on user query and context and relevance in a step by step manner in the way even a kid can understand
<context>
{context}
</contextt>
question: {input}
                                          '''

)

In [11]:
#select the llm model
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo")


In [16]:
#making a chain before hand where llm will be supplied with combination of (docs+userinput)known as prompyt

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)




In [17]:
#combine retriver and document chain which is like retrive the relavant info from db then put that info into llm in the form of prompt(which contains both relavant info and input query) thus known as retriever chain

from langchain.chains.retrieval import create_retrieval_chain

retrieval_chain =create_retrieval_chain(retriever, document_chain)


In [27]:
#run the entire chain in order
response =retrieval_chain.invoke({"input":"how to activate all chakras"})

In [1]:
response['answer']